## Preliminary Prediction Model

In [1]:
from pandas import read_csv
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

2023-02-03 15:22:26.110093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
tf.random.set_seed(7)

In [3]:
pred_seqs = pd.read_csv('pred_seqs.csv')
class_labels = pd.read_csv('class_labels.csv')

combined = pd.concat([pred_seqs['Prediction Sequences'], class_labels['Labels']], axis=1, keys=['Prediction Sequences', 'Labels'])

In [4]:
def clean_str(my_str):
    test_clean = my_str.replace('[', '').replace(']', '')
    test_arr = test_clean.split(' ')
    test_arr = list(filter(lambda x: x != '' and x != '...', test_arr))
    if len(test_arr) == 0:
        return np.array(test_arr)
    try:
        final_arr = [float(t)/100000 for t in test_arr]
    except Exception as e:
        print(e)
        print(test_arr)
        return
    return np.array(final_arr)

In [5]:
pred_seqs_2 = pred_seqs['Prediction Sequences'].apply(clean_str)

In [6]:
combined['Prediction Sequences'] = pred_seqs_2

In [15]:
combined['Labels'].value_counts()

0    1330908
1     370322
Name: Labels, dtype: int64

In [8]:
pred_time_seq_arr = np.array(combined['Prediction Sequences'], dtype=object)
labels_time_seq_arr = np.array(combined['Labels'], dtype=object).astype('float32')

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pred_time_seq_arr, labels_time_seq_arr, test_size=0.2, random_state=6013)

In [10]:
X_train = sequence.pad_sequences(X_train, maxlen=24, value=-1.)
X_test = sequence.pad_sequences(X_test, maxlen=24, value=-1.)

In [13]:
model = Sequential()
model.add(LSTM(50, input_shape = (24,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Epoch 1/3
21266/21266 [==============================] - 179s 8ms/step - loss: 0.1975 - accuracy: 0.9124 - val_loss: 0.1236 - val_accuracy: 0.9534
Epoch 2/3
21266/21266 [==============================] - 176s 8ms/step - loss: 0.1577 - accuracy: 0.9329 - val_loss: 0.1053 - val_accuracy: 0.9609
Epoch 3/3
21266/21266 [==============================] - 175s 8ms/step - loss: 0.1593 - accuracy: 0.9350 - val_loss: 0.1104 - val_accuracy: 0.9651
